In [18]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from torch.optim.optimizer import Optimizer
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
from transformers import pipeline
import warnings 
warnings.filterwarnings('ignore')
from pytorch_lightning import seed_everything
from torch.utils.data import DataLoader
import os
import gc
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForQuestionAnswering
import pandas as pd
from tqdm import tqdm_notebook as tqdm
gc.collect()

def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def set_seed(seed = int):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    seed_everything(seed)
    return random_state
random_state = set_seed(42)


model_path = '../models/Product_Data_SQuAD_model_product.pt'
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")
model.load_state_dict(torch.load(model_path))
model.eval()
nlp = pipeline('question-answering', model=model.to('cpu'), tokenizer=tokenizer)

def model_predict(nlp,df):
    table = pd.DataFrame()
    for i in tqdm(sorted(df.index.tolist())):
        sample = df.loc[[i]]
        string_X_train = sample['string_X_train'].values[0]
        QA_input = {
            'question': 'What is the product name?',
            'context': string_X_train
        }
        res = nlp(QA_input)
        predict = QA_input['context'][res['start']:res['end']]
        row = pd.DataFrame({'predict:':predict},index=[i])
        table = table.append(row)
    return table

def substringSieve(string_list):
    out = []
    for s in string_list:
        if not any([s in r for r in string_list if s != r]):
            out.append(s)
    return out

Global seed set to 42
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should pr

# 製作寶典

In [19]:
# 載入寶典
df = pd.read_excel('../data/寶典/寶典.v4.20211001.xlsx',engine='openpyxl')
df = df.rename(columns={'ITEMNM':'品名','DIVNM':'公司事業部門','CODIV':'公司代號'})
df['品名'] = df['品名'].apply(lambda x:x.strip())

# 載入產品csv
val_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv',index_col=0)[['string_X_train','Y_label','EXPNO','from']]
print(val_df.shape)

# 合併寶典和SPEC(後面四個月不要加)
n = -4
產品集合 = set(df['品名'].values.tolist() + val_df.loc[val_df['from'].isin(sorted(val_df['from'].unique())[:n]),'Y_label'].values.tolist())

# 對應表
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))

# 驗證集
val_df = val_df.loc[val_df['from'].isin(sorted(val_df['from'].unique())[-4:]),:]
print(val_df.shape)
display(val_df.head(3))

(11571, 4)
(3205, 4)


,string_X_train,Y_label,EXPNO,from
765,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx
766,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx
767,COMMODITY EVA TAISOX 7470M QUANTITY1 2500MTUN...,TAISOX 7470M,18,20210406.xlsx


# 如果品名是單詞的話 前後加個空白

In [20]:
新產品集合 = []
for p in 產品集合:
    if ' ' not in p: # 如果是單詞
        p = f' {p.strip()} ' # 前後加空白
        新產品集合.append(p) # append
    else:
        新產品集合.append(p) # append
產品集合 = list(set(新產品集合))

In [21]:
def Collection_method(df,產品集合):
    labels = {}
    for i in tqdm(df.index):
        products = []
        for p in 產品集合:
            if p in df.loc[i,'string_X_train']:
                products.append(p) # 加入候選清單
        labels[i] = products # 這是一個清單,含多個產品
    predict = pd.DataFrame(index=labels.keys(),columns=['predict'])
    predict['predict'] = list(labels.values())
    return predict
predict = Collection_method(val_df,產品集合)
result = val_df.join(predict)
result['class'] = 'rule'

In [22]:
result

,string_X_train,Y_label,EXPNO,from,predict,class
765,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx,"[BASE OIL, BASE OIL 500N]",rule
766,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx,"[BASE OIL, BASE OIL 500N]",rule
767,COMMODITY EVA TAISOX 7470M QUANTITY1 2500MTUN...,TAISOX 7470M,18,20210406.xlsx,"[ TAISOX , EVA , TAISOX 7470M]",rule
768,COMMODITY EVA TAISOX 7470M QUANTITY1 2500MTUN...,TAISOX 7470M,NaN,20210406.xlsx,"[ TAISOX , EVA , TAISOX 7470M]",rule
769,480 MT OF PVC RESIN GRADE S65 ATTHE RATE OF ...,GRADE,NaN,20210406.xlsx,"[ GRADE , RESIN , PVC RESIN]",rule
...,...,...,...,...,...,...
8203,1GPPS GP5250 QUANTITY200MT UNIT PRICEUSD13...,GP5250,NaN,202106.xlsx,[ GP5250 ],rule
8204,COMMODITY EVA TAISOX 7350M EVA TAISOX...,TAISOX 7470M,NaN,202106.xlsx,"[TAISOX 7350M, EVA TAISOX, EVA TAISOX 7350, T...",rule
8205,18 MT OF TAIRILAC ABS RESIN AT THE RATE USD 2...,ABS RESIN,NaN,202106.xlsx,"[ TAIRILAC , ABS RESIN, RESIN , ABS ]",rule
8206,COMMODITY TAIRILIN BRAND LOW...,GRADE,NaN,202106.xlsx,"[ TAIRILIN , GRADE ]",rule


In [23]:
not_find = []
for j,i in enumerate(result['predict'].values):
    if len(i) == 0:
        not_find.append(j)
len(not_find)

3

In [24]:
not_find_df = result.iloc[not_find]
not_find_df

,string_X_train,Y_label,EXPNO,from,predict,class
5205,18 MTS POM FORMOCON FM130 UNIT PRICE USD 225...,FM130,1P,202104.xlsx,[],rule
5335,TERMS OF PRICE FOB KAOHSIUNG TAIWAN COUNTRY O...,YUNGSOX 1040F,1P,202104.xlsx,[],rule
6007,TERMS OF PRICE FOB KAOHSIUNG PORT OF TAIWANCO...,YUNGSOX 2080,1P,202104.xlsx,[],rule


In [25]:
bert_predict = model_predict(nlp,not_find_df)
bert_predict

,predict:
5205,FM130
5335,YUNGSOX 1040F
6007,YUNGSOX 2080


In [26]:
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [27]:
if len(not_find) > 0:
    idx = not_find_df.index
    result.loc[idx,'predict'] = [ [str(i)] for i in bert_predict['predict:'].values]
    result.loc[idx,'class'] = 'bert'
    display(result.loc[idx,:])

,string_X_train,Y_label,EXPNO,from,predict,class
5205,18 MTS POM FORMOCON FM130 UNIT PRICE USD 225...,FM130,1P,202104.xlsx,[FM130],bert
5335,TERMS OF PRICE FOB KAOHSIUNG TAIWAN COUNTRY O...,YUNGSOX 1040F,1P,202104.xlsx,[YUNGSOX 1040F],bert
6007,TERMS OF PRICE FOB KAOHSIUNG PORT OF TAIWANCO...,YUNGSOX 2080,1P,202104.xlsx,[YUNGSOX 2080],bert


In [28]:
lst = []
for i in result.iloc[not_find].index:
    if result.loc[i,'Y_label'] != result.loc[i,'predict']:
        lst.append(i)

In [29]:
def get_acc(df,t=0.75):
    correct = []
    correct_label = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        if max(jacs) >= t:
            correct.append('yes')
        else:
            correct.append('no')
    result = pd.DataFrame({'correct':correct})
    return result['correct'].value_counts()['yes']/len(result)

def get_jac(df):
    all_jacs = []
    for i in df.index:
        jacs = []
        for j in df.loc[i,'predict']:
            jacs.append(get_jaccard_sim(df.loc[i,'Y_label'],j))
        all_jacs.append(max(jacs))
    return np.sum(all_jacs)/len(all_jacs)

# 表現

In [30]:
get_acc(result,t=1),get_acc(result,t=0.75),get_jac(result)

(0.9940717628705148, 0.9940717628705148, 0.9952678107124284)

In [31]:
部門_lst = []
for p_lst in tqdm(result['predict'].values):
    p = max(p_lst,key=len)
    jac_dict = {}
    for i in 品名2代號.keys():
        jac_dict[i] = get_jaccard_sim(i,p)
    部門_lst.append(品名2代號[max(jac_dict, key=jac_dict.get)])
result['預測部門代號'] = 部門_lst
result['predict'] = [substringSieve(i) for i in result['predict']]
result['EXPNO'] = [ str(i).strip() for i in result['EXPNO']]
result = result[result['EXPNO']!=str(np.nan)]
result['預測部門代號'] = [ str(i).strip() for i in result['預測部門代號']]
display(result)
a = len(result[result['EXPNO']==result['預測部門代號']])
b = len(result[result['EXPNO']!=result['預測部門代號']])
print(f'部門預測正確數量:{a} 錯誤數量:{b} 正確率:{a/(a+b)}')

,string_X_train,Y_label,EXPNO,from,predict,class,預測部門代號
765,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx,[BASE OIL 500N],rule,60
766,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx,[BASE OIL 500N],rule,60
767,COMMODITY EVA TAISOX 7470M QUANTITY1 2500MTUN...,TAISOX 7470M,18,20210406.xlsx,"[ TAISOX , EVA , TAISOX 7470M]",rule,18
770,EVA TAISOX 7470M QUANTITY 10000MTS UNIT ...,TAISOX 7470M,18,20210406.xlsx,"[TAISOX 7350M, EVA TAISOX 7350, TAISOX 7470M]",rule,18
771,EVA TAISOX 7470M QUANTITY 10000MTS UNIT ...,TAISOX 7470M,18,20210406.xlsx,"[TAISOX 7350M, EVA TAISOX 7350, TAISOX 7470M]",rule,18
...,...,...,...,...,...,...,...
8168,ITEM 1180 METRIC TONS OF ABS RESIN GRADE NO ...,GRADE,4A,202106.xlsx,"[ GRADE , ABS RESIN, RESIN , ABS ]",rule,4A
8180,ITEM1 144 METRIC TONS OF ABS RESIN GRADE NO ...,GRADE,4A,202106.xlsx,"[ PS , PS RESIN, GRADE , ABS RESIN, RESIN , ...",rule,4A
8181,ITEM1 144 METRIC TONS OF ABS RESIN GRADE NO ...,GRADE,4A,202106.xlsx,"[ PS , PS RESIN, GRADE , ABS RESIN, RESIN , ...",rule,4A
8198,COMMODITY MONO ETHYLENE GLYCOL MEGQUANTITY 4...,MONO ETHYLENE GLYCOL,2A,202106.xlsx,[MONO ETHYLENE GLYCOL],rule,2A


部門預測正確數量:2183 錯誤數量:428 正確率:0.8360781309842972


In [32]:
result[result['EXPNO'] != result['預測部門代號']]

,string_X_train,Y_label,EXPNO,from,predict,class,預測部門代號
765,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx,[BASE OIL 500N],rule,60
766,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx,[BASE OIL 500N],rule,60
833,PROCESS OIL 380N 60MT 5 PCT AT ICIS PRICING ...,PROCESS OIL 380N,61,20210406.xlsx,[PROCESS OIL],rule,64
861,30 MTONS BISPHENOLA PACKED IN 750 KGS BAGS A...,BISPHENOLA,2P,20210406.xlsx,[ BISPHENOLA ],rule,4A
876,TERMS OF PRICE CIF BUSAN SOUTH KOREACOUNTRY O...,YARN,XU,20210406.xlsx,"[ GRADE , YARN ]",rule,4A
...,...,...,...,...,...,...,...
8120,PLASTIC RESIN CIF HAIPHONG VIETNAM1 1 X 20 FCL...,PP 5090T,1P,202106.xlsx,"[ PP , PP 5090T, PLASTIC RESIN, RESIN ]",rule,23
8124,COMMODITY PURIFIED ISOPHTHALIC ACID PIAQUANT...,ACID,48,202106.xlsx,[ ACID ],rule,1A
8141,COMMODITY PURIFIED TEREPHTHALIC ACID PTA QU...,ACID,48,202106.xlsx,"[ PTA , ACID ]",rule,1A
8142,COMMODITY PURIFIED TEREPHTHALIC ACID PTA QU...,ACID,48,202106.xlsx,"[ PTA , ACID ]",rule,1A


# 一些小問題 公司代號跟EXPNO對不上

In [33]:

table1 = pd.DataFrame()
table2 = pd.DataFrame()
p = result[result['EXPNO'] != result['預測部門代號']]['predict'].values[0][0]
for p in list(set(品名2代號.keys())&set(result['Y_label'])):
    if 品名2代號[p] != result.loc[result['Y_label']==p,'EXPNO'].values[0]:
        table1 = table1.append(df[df['品名']==p])
        table2 = table2.append(val_df[val_df['Y_label']==p])
display(table1.drop_duplicates(subset=['品名'])),display(table2.drop_duplicates(subset=['Y_label']))

,RIGID,EXTN,公司代號,公司事業部門,品名
339,NaN,NaN,60,國外處,BASE OIL 150N
54,黃淑誼,8405,4A,台化塑膠事業部,PC RESIN
227,NaN,NaN,25,纖維事業部,RELEASE FILM
340,NaN,NaN,60,國外處,BASE OIL 500N
881,耿瑞君,8215,22,南亞塑一部二處,SYNTHETIC PAPER
902,葉鳳琴,8233,23,南亞塑三部,PP SHEET


,string_X_train,Y_label,EXPNO,from
996,BASE OIL 150N PRICING1000MT5 PCT AT ICIS PRIC...,BASE OIL 150N,61,20210406.xlsx
7747,DESCRIPTION QUANTITY UNIT PRICE ...,PC RESIN,4A,202106.xlsx
5015,TERMS OF PRICE FOB TAIWAN PORTCOUNTRY OF ORIG...,RELEASE FILM,2E,202104.xlsx
765,1 GROUP II 500N BASE OIL 500N QUANTITY 900...,BASE OIL 500N,61,20210406.xlsx
5009,1PP SYNTHETIC PAPER BJE OF5 200UM X 1054MM ...,SYNTHETIC PAPER,22,202104.xlsx
940,1 100000 KGS OF PP SHEET RIGID PVC SHEET A...,PP SHEET,22,20210406.xlsx


(None, None)

In [35]:
result.to_csv('submit_product_規則加bert.csv')